In [ ]:
# Here we consolidate functions across multiple versions of 123NF driver into one driver file

In [ ]:
import importlib
import setup_nx # your own module, setup.nx.py
import numpy as np
import math as m
import statistics as st
import cmath
import matplotlib.pyplot as plt 
import itertools
import random
from operator import add
importlib.reload(setup_nx)
from setup_nx import *
from graphviz import Source, render
import pydot

import datetime
import time

import my_feeder_funcs as ff
import my_impedance_funcs as imp
import my_configVis_funcs as vis
import my_detControlMatExistence_funcs as ctrl
import my_detLznRange_funcs as lzn
import my_heatmapSetup_funcs as hm

In [ ]:
# [Essential] specify input feeder data

#-----------------------------------------------------------------
'specifying file paths'
# Enter the path/name of the model's excel file and import
    # All GridBright load files should be in the following folder
    #loadfolder = "/Users/jasperpakshong/Documents/Berkeley/ENERGISE/IEEE13/"
    #loadpath = loadfolder + "IEEE13testload_w_extreme_act.xlsx"

    # filepath = "IEEE13/"
    # modelpath = filepath + "001 phasor08_IEEE13_OPAL.xls"
    # loadfolder = "IEEE13/"
    # loadpath = loadfolder + "001_phasor08_IEEE13_norm03_HIL_7_1.xlsx"

    #filepath = "AL0001/"
    #modelpath = filepath + "AL0001_OPAL_working.xls"
    #loadfolder = "AL0001/"
    #loadpath = loadfolder + "AL0001_tvload_afternoon1h.csv"
      
    #filepath = "13NF_balanced/"
    #modelpath = filepath + "016 GB_IEEE13_balance_all_ver2.xls"
    #loadfolder = "13NF_balanced/"
    #loadpath = loadfolder + "016 GB_IEEE13_balance all ver2_time_sigBuilder_secondWise_norm03.csv"

filepath = "123NF/"
modelpath = filepath + "004_GB_IEEE123_OPAL.xls"
loadfolder = "123NF/"
loadpath = loadfolder + '004_123NF_PVpen100_nocloud_minutewise_whead.csv'
headerpath = '123NF/004_GB_IEEE123_time_header.csv'
load_data = loadpath


#==========================================================================================================

'specifying file name'
#file_name = string specifying name of dot file created when make_graph() is called
file_name = '123NF_test.dot'

#==========================================================================================================

'Specify substation kV, kVA bases, name, and the number of timesteps in the load data'
Vbase_ll = 4160
Vbase = Vbase_ll / np.sqrt(3)
Sbase = 5000/3
substation_name = 'bus_150'
timesteps = 1

'DO NOT NEED TO EDIT THIS CELL BEFORE RUNNING'

ts = time.time()
print()
print(datetime.datetime.fromtimestamp(ts))

plot = 0 #turn plot on/off

depths = {}
leaves = []

In [ ]:
# [ESSENTIAL] create feeder obj
fin_feeder = ff.feeder_init(modelpath,loadfolder,loadpath,timesteps,Vbase_ll,Sbase,depths,leaves)
print("Finished initializing feeder")
ff.make_graph(fin_feeder, file_name)
node_index_map = hm.createNodeIndexMap(fin_feeder) #node indices for indicMat and F matrix
R,X=hm.createRXmatrices_3ph(fin_feeder, node_index_map,depths)

print('depths=',depths) # should be populated

count = 0 # print list of buses in network
for i in fin_feeder.network:    
    print(i) 
    count += 1
    if count >= 10:
        break
    
    
import csv 
graph = fin_feeder.network
#print(graph.nodes)

# write busnames into a csv
# with open("123NF_busList.csv", 'w', newline='') as csvfile:
#     spamwriter = csv.writer(csvfile, delimiter='-',
#                             quotechar='|', quoting=csv.QUOTE_MINIMAL)
#     spamwriter.writerows(graph.nodes)
    
Source.from_file(file_name)
#^ need this to plot feeder

In [ ]:
# [Optional] run impedance-related functions
slack_bus = None
for bus_name, depth in depths.items():
    if depth == 0:
        slack_bus = bus_name
        break
        
# -------------------- now we call functions: ---------------------------------------------
#print(depths)
# modify node names when change feeders
#plot_histogram_RX_ratios(fin_feeder, leaves_only = True)
print('Z between buses:')
print(np.around(imp.get_total_impedance_between_two_buses(fin_feeder, 'bus_37', 'bus_15',depths),2))
print('\nBus 49 Z to substation:')
print(np.around(imp.get_total_impedance_from_substation(fin_feeder, 'bus_49',depths),2))
print('\nR/X ratio of bus 49 to substation:')
print(imp.get_RX_ratio_tosubst(fin_feeder,'bus_49',depths))
# should check on how to format the printing do that it's to like 2 decimal places

print('R/X Ratios:')
print('\n67 to 79: ')
print(imp.get_RX_ratio_between_two_buses(fin_feeder, 'bus_67', 'bus_79',depths))
print('\n67 to 96: ')
print(imp.get_RX_ratio_between_two_buses(fin_feeder, 'bus_67', 'bus_96',depths))
print('\n67 to 104: ')
print(imp.get_RX_ratio_between_two_buses(fin_feeder, 'bus_67', 'bus_104',depths))
print('\n67 to 114: ')
print(imp.get_RX_ratio_between_two_buses(fin_feeder, 'bus_67', 'bus_114',depths))
print('\n57 to 66: ')
print(imp.get_RX_ratio_between_two_buses(fin_feeder, 'bus_57', 'bus_66',depths))
print('\n18 to 251: ')
print(imp.get_RX_ratio_between_two_buses(fin_feeder, 'bus_18', 'bus_251',depths))
print('\n35 to 151: ')
print(imp.get_RX_ratio_between_two_buses(fin_feeder, 'bus_35', 'bus_151',depths))

In [ ]:
# Print 3-phase node_index table
my_buses=list(node_index_map.keys())
my_list=list(node_index_map.values())
list1 = [(i+1) * 3-2 for i in my_list]
list2 = [(i+1) * 3-1 for i in my_list]
list3 = [(i+1) * 3-0 for i in my_list]
for my_buses,list1, list2,list3 in zip(my_buses,list1,list2,list3):
    print(my_buses,list1, list2,list3)
#Table format: [bus_name, phaseA idx, phaseB idx, phaseC idx]
# indices are shifted by 1 to be for MATLAB (starts at 1 not 0)

# R is of size 387, which is 129*3, so set n=129
n=129
A, B = hm.setupStateSpace(n,fin_feeder, node_index_map,depths)
print('R = '+str(R))
print('X = '+str(X))
print('A = '+str(A))
print('B = '+str(B))
print(A.shape)
print(B.shape)

# write A and B matrices to csv
#np.savetxt("123NF_Amat.csv", A, delimiter=",")
#np.savetxt("123NF_Bmat.csv", B, delimiter=",")

# Save R and X matrices to csv to import into matlab
# np.savetxt reference: https://thispointer.com/how-to-save-numpy-array-to-a-csv-file-using-numpy-savetxt-in-python/
#np.savetxt('Rmat_123NF.csv', R, delimiter=',')
#np.savetxt('Xmat_123NF.csv', X, delimiter=',')

In [ ]:
# ------- if need to convert .dot to .png ------------
(graph,) = pydot.graph_from_dot_file('heat_map_6_123NF_test.dot')
graph.write_png('hm_nbhd_6_test.png')
#Source.from_file('heat_map_0_123NF_test.dot')

In [ ]:
# Create Fig 1: # find_good_colocated colormap

# all_act_locs = []
# perf_nodes = []
# # empty act locs asks heatMapProcess to create heatmap for empty network, i.e. feas of placing 1 co-located actuator at each loc
# feas_configs, heatMapNames=hm.find_good_colocated(fin_feeder, [], node_index_map,substation_name,depths, file_name)
# print('----------- Feas configs are: ---------')
# print(feas_configs)
# Source.from_file(heatMapNames[0]) # display graph in notebook

In [ ]:
# Create figure 2, colormap of phase coupling
ratios = vis.phaseCouplingPerNode(fin_feeder,depths,file_name)
vis.createColorMap(fin_feeder, ratios, 'fig2_colorMap_123NF')
Source.from_file('fig2_colorMap_123NF') # display graph in notebook
ff.clear_graph(fin_feeder)

In [ ]:
# ----- Run computeFParamSpace ---------
# all_act_locs = ['bus_25','bus_39','bus_56','bus_108'] # 49, then 76, then 65
# perf_nodes = ['bus_25','bus_39','bus_56','bus_108']


# Fq_ub,Fp_ub=ctrl.computeFParamSpace_v2(fin_feeder, all_act_locs, perf_nodes,R,X,depths,node_index_map)
# # this function prints Zgood (from R and X matrices) and Z_toSubst, where Z_toSubst is correct but doesnt match Zgood. Investigate
# print('(Fp_ub,Fq_ub)=(',Fp_ub,',',Fq_ub,')')

#vis.markActuatorConfig(all_act_locs, fin_feeder, file_name)

In [ ]:
# --------- Config Chunk (Assign act and perf locs here) ----------

# binary heat map that shows good locations to place a new actuator when
# a) there are 0 existing actuators, b) there is 1 existing actuator, and c) there are two existing actuators.
# actuator list = complex config 1, i.e. at each step we choose one of the good places and buildup to complex config 1

# #complex config 1 (feas)
#all_act_locs = ['bus_49','bus_76','bus_65'] # 49, then 76, then 65
#perf_nodes = ['bus_49','bus_76','bus_65']
# 16336 seconds 

#complex config 2 (feas)
# all_act_locs = ['bus_25','bus_39','bus_56','bus_108'] # 49, then 76, then 65
# perf_nodes = ['bus_25','bus_39','bus_56','bus_108']
# 30206 seconds

# #complex config 3 (feas)
# all_act_locs = ['bus_105','bus_112','bus_100','bus_91'] # 49, then 76, then 65
# perf_nodes = ['bus_105','bus_112','bus_100','bus_91']
# # time taken = 21674 seconds

# #complex config 4, many act 1 perf
#all_act_locs = ['bus_42','bus_42','bus_42','bus_42'] # 49, then 76, then 65
#perf_nodes = ['bus_151','bus_46','bus_251','bus_8']
# # time taken = XXX seconds

# # complex config 3
# all_act_locs = ['bus_105','bus_112','bus_100','bus_91'] # 49, then 76, then 65
# perf_nodes = ['bus_105','bus_112','bus_100','bus_91']

# evaluate this config
# all_act_locs = ['bus_60','bus_58','bus_20','bus_72'] # 49, then 76, then 65
# perf_nodes = ['bus_300','bus_58','bus_20','bus_72']

# Adam sim
# all_act_locs = ['bus_49','bus_76','bus_152'] # 49, then 76, then 65
# perf_nodes = ['bus_49','bus_76','bus_65']

# # eval neighborhood config
# all_act_locs = ['bus_82','bus_87','bus_76','bus_49','bus_41','bus_46','bus_60'] # 49, then 76, then 65
# perf_nodes = ['bus_77','bus_77','bus_77','bus_44','bus_44','bus_44','bus_66']

# run RHP on neighborhood config
# heatmap/collect stable configs for final step only:
# set_act_locs = ['bus_82','bus_87','bus_76','bus_49','bus_41','bus_46']
# set_perf = ['bus_77','bus_77','bus_77','bus_44','bus_44','bus_44']
# addon_locs = ['bus_60']
# addon_perf_nodes = ['bus_66']
# heatmap/collect stable configs for all steps
set_act_locs = []
set_perf = []
addon_locs = ['bus_82','bus_87','bus_76','bus_49','bus_41','bus_46','bus_66']
addon_perf_nodes = ['bus_77','bus_77','bus_77','bus_44','bus_44','bus_44','bus_66']


# # run RHP for step 3 of 10-step placeMaxColoc, not going in paper
# set_act_locs = ['bus_58','bus_20','bus_72']
# set_perf = ['bus_58','bus_20','bus_72']
# addon_locs = ['bus_300']
# addon_perf_nodes = ['bus_300']

# motivating example (motex), list is equal to output 'max_act_config_stopInfeas'
# all_act_locs=['bus_55','bus_33','bus_5','bus_96','bus_113','bus_197']
# perf_nodes=['bus_55','bus_33','bus_5','bus_96','bus_113','bus_197']

# set_act_locs = ['bus_55','bus_33','bus_5','bus_96','bus_113','bus_197']
# set_perf = ['bus_55','bus_33','bus_5','bus_96','bus_113','bus_197']
# addon_locs = ['bus_60']
# addon_perf_nodes = ['bus_60']

In [ ]:
# ---------- mark actuator config -------------
# assign 'all_act_locs' in config chunk
vis.markActuatorConfig(all_act_locs,fin_feeder, file_name)
(graph,) = pydot.graph_from_dot_file('actConfig_123NF_test.dot') # requires 'import pydot'
graph.write_png('actConfig_123NF_test.png')

In [ ]:
# --------- run eval_config, define config above^ --------------
# assign 'all_act_locs' and 'perf_nodes' in config chunk
# feas, maxError,numfeas=hm.eval_config(fin_feeder, all_act_locs,perf_nodes, node_index_map,substation_name,depths,file_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)

In [ ]:
# -------- run eval_config, after trying to place actuators at ALL nodes in graph list -----------
# so far single perfs at these locs work: 46,300,84,18
# hypothesis: multiple actautors on every node tracking single perf is feas for ANY choice of perf

# assign 'all_act_locs' and 'perf_nodes' in config chunk
graphNodes_nosub = hm.remove_subst_nodes(fin_feeder, file_name) # dont consider co-located at substation nodes, node 650 and 651
all_act_locs=graphNodes_nosub[:] # place actuators at ALL non-substation nodes
n=len(all_act_locs)
print('acts=',all_act_locs)

perf='bus_84' # you choose
perf_nodes=[perf] * n
print('perfs=',perf)
feas, maxError,numfeas=hm.eval_config(fin_feeder, all_act_locs,perf_nodes, node_index_map,substation_name,depths,file_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)

In [ ]:
# ------------ run placeMaxColocActs_stopAtInfeas -----------
# place colocated actuators until an infeasible loc is tested, then call find_good_colocated and return 
# produces colormap associated with "motex" motivating example

# t = time.time()
# max_act_config_stopInfeas=hm.placeMaxColocActs_stopAtInfeas(fin_feeder, file_name, node_index_map, depths, substation_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)
# elapsed = time.time() - t
# print('Time elapsed=',elapsed)

In [ ]:
# ------ Place place_max_colocated_acts -------------
t = time.time()
max_act_config = hm.place_max_colocated_acts(fin_feeder, file_name, node_index_map, depths, substation_name)
elapsed = time.time() - t
print('Time elapsed=',elapsed)

In [ ]:
# ------- run runHeatMapProcess (RHP) ---------------
# assign 'set_act_locs, set_perf, addon_locs, addon_perf_nodes' in config chunk

t = time.time()
# set_act_locs and set_perf are existing pairs, addon_locs and addon_perf_nodes are pairs to add after produce heatmap
lst_feas_configs, lzn_error_run_sum, heatMapNames = hm.runHeatMapProcess(fin_feeder, set_act_locs, set_perf, addon_locs, addon_perf_nodes, node_index_map, substation_name, depths, file_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)
# lst_feas_configs is list of dictionaries
elapsed = time.time() - t
print('Time elapsed=',elapsed)

In [ ]:
# # Save data 'lst_feas_configs' to .pkl file
# import pickle
# filename='nbhs_RHP_lstfeasconfigs.pkl'
# with open(filename, 'wb') as f:  # Python 3: open(..., 'wb')
#     pickle.dump(lst_feas_configs, f)

In [ ]:
# load data 'lst_feas_configs' from .pkl file
import pickle
filename='nbhs_RHP_lstfeasconfigs.pkl'
with open(filename,'rb') as f:  # Python 3: open(..., 'rb')
    lst_feas_configs = pickle.load(f)

In [ ]:
# to save and restore variables..
# https://stackoverflow.com/questions/2960864/how-can-i-save-all-the-variables-in-the-current-python-session

In [ ]:
# ---------- Setup branch analysis on 'lst_feas_configs' populated by RHP -------------

In [ ]:
# ------------ create histogram of numfeas --------------
numFeasList=[]
#print([d['numfeas'] for d in lst_feas_configs])
for cfg in lst_feas_configs:
    a=cfg['numfeas']
    b=a[0].item() # remove layers of brackets
    numFeasList.append(b)

print(numFeasList)
plt.title("numFeas Histogram")
plt.xlabel("number of feasible Fs")
plt.ylabel("number of configs")
plt.hist(numFeasList, bins=40) # -10: is last 10 ele
#plt.hist(numFeasList[-10:], bins=40) # -10: is last 10 ele
plt.savefig('numFeasHist.png') # modify for each feeder
plt.show() # need to savefig before plt.show

print(len(numFeasList))

In [ ]:
# convert lst_feas_configs (list of dictionaries) to newlst_feas_configs (a list of lists)
# input a list of feasible actuator configurations where each configuration is its own list (ie a list of lists)"
length=len(lst_feas_configs)
newlst_feas_configs = []
numact_lst=[] # store number of actuators in separate list
i=0
for dic in lst_feas_configs:
    #print('dic=',dic)
    #print(type(dic))
    i+=1
    if i<length-1:
        #print('dic[act]=',str(dic['act']))
        newlst_feas_configs += [dic['act']]
        numact_lst+=[dic['numfeas']]

In [ ]:
# create 'placed' variable
# placed is an ordered list of lists, where the k'th ele of the list is the actuators placed at that step
set_act_locs = []
set_perf = []
addon_locs = ['bus_82','bus_87','bus_76','bus_49','bus_41','bus_46','bus_66']
addon_perf_nodes = ['bus_77','bus_77','bus_77','bus_44','bus_44','bus_44','bus_66']

acts=set_act_locs + addon_locs
placed=[]
k=0
while k<=len(acts):
    placed+=[acts[0:k]]
    k+=1
print(placed)

In [ ]:
# ----- run plot_actuator_num_histogram ---------"
# input list of lists of feasible actuator configurations"
# output histogram reflecting number of actuators in each feas configuration"
vis.plot_actuator_num_histogram(newlst_feas_configs, file_name)  
# note the max number in the histogram

# ----- run assign_network_branches ---------"
branches = vis.assign_network_branches(fin_feeder, substation_name)
vis.mark_network_branches(fin_feeder, branches, file_name, substation_name,depths)
Source.from_file('branch_key:' + file_name)
print('branches table:') # format into a table with index and branch nodes
i=0
for br in branches:
    print(i,',',br)
    i+=1


# ----- run find_good_branches ---------"
# the 'best' branch is the branch with the most actuators from feas configs on it across all configurations
num_good_branches = 8 # return the n best branches
vis.find_good_branches(newlst_feas_configs, branches, num_good_branches, placed)

# ----- run determine_good_or_bad_branch ---------"
#print('\nBranches are ', branches)
branch = branches[-2]  # you choose as input
print('\nBranch chosen=',branch)
good_branch_threshold = 5 # threshhold # configs that must use the branch (i.e. an act on that branch) for the branch to be "good"
configs_with_branch = vis.determine_good_or_bad_branch(branch, newlst_feas_configs, good_branch_threshold)
# output: configs that use the branch, and if that # exceeds threshhold, call the branch "good"
print('Num feas configs =', len(newlst_feas_configs))